Příznaky
=====


Strojové učení
-------------

*Features are workhorses of machine learning. (Peter Flach: Machine learning)*


![mlscheme](mlscheme.png "Základní koncepty strojovéo učení ([1]).")

- __Úloha:__ problém, který se řeší strojovým učením 
- __Model:__ výstup strojového učení 
- __Učící algoritmus:__ způsob tvorby modelu
- __Příznaky:__ vstupy modelu

Příznak (feature) lze chápat jako jakoukoliv hodnotu, kterou získáme z instancí problému ML.
Zobrazení z prostoru instancí do domény příznaku (hodnot).

Kvalita příznaků (atributů) ovlivňuje úspěch strojového učení.

Typy
----


Příklady: reálná čísla, celá čísla (počet), Boolean (příznak je-není), konečné množiny (barvy ...)

Tradiční dělení v ML:

- __kvantitativní (číselné)__
mají rozsah, jsou uspořádané, 
někdy se jim (nepřesně) říká spojité, 
lze u nich měřit řadu (statistických) charakteristik

- __ordinální__
uspořádané ale bez měřítka

- __kategoriální (nominální, diskrétní)__
neuspořádané


> Přemýšlejme: věk člověka v letech, vs. číslo popisné

Statistiky
---------

- centrální tendence

  - průměr (střední hodnota)

  - medián

  - modus - lze určit u všech typů příznaků

medián "obvykle" leží mezi průměrem a modem (K. Pearson - do 1/3 od průměru k modu) 


- rozložení

  - rozptyl - střední hodnota kvadrátů odchylek od střední hodnoty

  - směrodatná odchylka -  kvadratický průměr odchylek hodnot znaku od jejich aritmetického průměru. neboli odmocnina z rozptylu, je ve stejných jednotkách jako příznak

  - kvantily - čísla (hodnoty), která dělí soubor seřazených (například naměřených) hodnot na několik zhruba stejně velkých částí

> Přemýšlejme: 50. percentil a 5. decil a druhý kvartil jsou ...  

- tvar

momenty, histogram (viz minule)

Modely vs příznaky
-----------------

Modely jako klasifikační stromy rozdělují kategoriální veličiny podle hodnot, u ordinálních a kvantitativních typicky dělají binární split. Na rozsahu hodnot nezáleží. 

To samé dělají modely založené na pravidlech.

Modely jako naivní Bayesův klasifikátor naopak pracují s kategoriálními veličinami, ordinální převedou na kategoriální, číselné je třeba diskretizovat.

Velká (největší?) řada metod CI naopak pracuje pouze s kvantitativními příznaky a ostatní na ně převádí.

Transformace příznaků
--------------------

|                 | kvantitativní | ordinální | kategoriální | Boolean |
|-----------------|---------------|-----------|--------------|---------|
| kvantitativní | normalizace | kalibrace | kalibrace | kalibrace |
| ordinální | diskretizace | seřazení | seřazení |seřazení |
| kategoriální | diskretizace | odřazení | grouping | - |
| Boolean | thresholding | thresholding | binarizace | - |


__Unsupervised thresholding__ - spočítat medián, průměr jako split-point

__Supervised thresholding__ - když víme kategorii apod, lze split optimalizovat


__Diskretizace__ - opět binning nebo s učitelem (bottom-up, rekurzivně top-down)

__Normalizace příznaku__ - centrovat průměrem, dělit rozptylem (normalita), centrovat medianem, dělit interkvartilovou vzdáleností

__Kalibrace příznaku__ - změnit měřitko příznaku tak, aby odráželo informace o výsledné třídě (tedy supervised) - např. u binární klasifikace škálujeme příznak pomocí pravděpodobnosti pozitivní třídy v závislosti na konkrétní hodnotě příznaku. Mnoha modelům to pomůže.

> Přemýšlejme: pro kategoriální příznaky jenom spočítáme relativní frekvence dle tříd v datech.


In [9]:
>>> measurements = [
...     {'city': 'Dubai', 'temperature': 33.},
...     {'city': 'London', 'temperature': 12.},
...     {'city': 'San Fransisco', 'temperature': 18.},
... ]

>>> from sklearn.feature_extraction import DictVectorizer
>>> vec = DictVectorizer()

>>> arr = vec.fit_transform(measurements).toarray()
print (arr)
>>> vec.get_feature_names()
['city=Dubai', 'city=London', 'city=San Fransisco', 'temperature']

[[  1.   0.   0.  33.]
 [  0.   1.   0.  12.]
 [  0.   0.   1.  18.]]


['city=Dubai', 'city=London', 'city=San Fransisco', 'temperature']

In [1]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

iris = load_iris()

X, y = iris.data, iris.target
pca = PCA(n_components=2)
selection = SelectKBest(k=1)
combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
X_features = combined_features.fit(X, y).transform(X)

svm = SVC(kernel="linear")
pipeline = Pipeline([("features", combined_features), ("svm", svm)])

param_grid = dict(features__pca__n_components=[1, 2, 3],
                  features__univ_select__k=[1, 2],
                  svm__C=[0.1, 1, 10])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10)
grid_search.fit(X, y)
print(grid_search.best_estimator_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] features__univ_select__k=1, features__pca__n_components=1, svm__C=0.1 
[CV]  features__univ_select__k=1, features__pca__n_components=1, svm__C=0.1, score=0.960784 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=1, svm__C=0.1 
[CV]  features__univ_select__k=1, features__pca__n_components=1, svm__C=0.1, score=0.901961 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=1, svm__C=0.1 
[CV]  features__univ_select__k=1, features__pca__n_components=1, svm__C=0.1, score=0.979167 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=1, svm__C=1 
[CV]  features__univ_select__k=1, features__pca__n_components=1, svm__C=1, score=0.941176 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=1, svm__C=1 
[CV]  features__univ_select__k=1, features__pca__n_components=1, svm__C=1, score=0.921569 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=1

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  12 tasks       | elapsed:    0.1s



[CV] features__univ_select__k=2, features__pca__n_components=1, svm__C=0.1 
[CV]  features__univ_select__k=2, features__pca__n_components=1, svm__C=0.1, score=0.960784 -   0.0s
[CV] features__univ_select__k=2, features__pca__n_components=1, svm__C=0.1 
[CV]  features__univ_select__k=2, features__pca__n_components=1, svm__C=0.1, score=0.921569 -   0.0s
[CV] features__univ_select__k=2, features__pca__n_components=1, svm__C=0.1 
[CV]  features__univ_select__k=2, features__pca__n_components=1, svm__C=0.1, score=0.979167 -   0.0s
[CV] features__univ_select__k=2, features__pca__n_components=1, svm__C=1 
[CV]  features__univ_select__k=2, features__pca__n_components=1, svm__C=1, score=0.960784 -   0.0s
[CV] features__univ_select__k=2, features__pca__n_components=1, svm__C=1 
[CV]  features__univ_select__k=2, features__pca__n_components=1, svm__C=1, score=0.921569 -   0.0s
[CV] features__univ_select__k=2, features__pca__n_components=1, svm__C=1 
[CV]  features__univ_select__k=2, features__pca_

[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  24 tasks       | elapsed:    0.3s



[CV] features__univ_select__k=1, features__pca__n_components=2, svm__C=1 
[CV]  features__univ_select__k=1, features__pca__n_components=2, svm__C=1, score=0.979167 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=2, svm__C=10 
[CV]  features__univ_select__k=1, features__pca__n_components=2, svm__C=10, score=0.980392 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=2, svm__C=10 
[CV]  features__univ_select__k=1, features__pca__n_components=2, svm__C=10, score=0.941176 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=2, svm__C=10 
[CV]  features__univ_select__k=1, features__pca__n_components=2, svm__C=10, score=0.979167 -   0.0s
[CV] features__univ_select__k=2, features__pca__n_components=2, svm__C=0.1 
[CV]  features__univ_select__k=2, features__pca__n_components=2, svm__C=0.1, score=0.980392 -   0.0s
[CV] features__univ_select__k=2, features__pca__n_components=2, svm__C=0.1 
[CV]  features__univ_select__k=2, features__pca_

[Parallel(n_jobs=1)]: Done  31 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  40 tasks       | elapsed:    0.4s



[CV] features__univ_select__k=1, features__pca__n_components=3, svm__C=0.1 
[CV]  features__univ_select__k=1, features__pca__n_components=3, svm__C=0.1, score=0.941176 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=3, svm__C=0.1 
[CV]  features__univ_select__k=1, features__pca__n_components=3, svm__C=0.1, score=0.979167 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=3, svm__C=1 
[CV]  features__univ_select__k=1, features__pca__n_components=3, svm__C=1, score=1.000000 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=3, svm__C=1 
[CV]  features__univ_select__k=1, features__pca__n_components=3, svm__C=1, score=0.941176 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=3, svm__C=1 
[CV]  features__univ_select__k=1, features__pca__n_components=3, svm__C=1, score=0.979167 -   0.0s
[CV] features__univ_select__k=1, features__pca__n_components=3, svm__C=10 
[CV]  features__univ_select__k=1, features__pca__n_

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:    0.5s finished
